In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
weather_csv_file = "../output_data/cities.csv"

starter_data = pd.read_csv(weather_csv_file)

starter_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [20]:
gmaps.configure(api_key=g_key)

weather_data = starter_data.dropna()
humidity = weather_data["Humidity"].astype(float) 
locations = weather_data[["Lat", "Lng"]].astype(float)

humid = gmaps.figure()
humid_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_data["Humidity"]),
                                 point_radius = 3)
humid.add_layer(humid_layer)

humid

Figure(layout=FigureLayout(height='420px'))

In [11]:
temp_filter = weather_data.loc[(weather_data["Max Temp"] < 80) & (weather_data["Max Temp"] > 70)]
wind_filter = temp_filter.loc[temp_filter["Wind Speed"] < 10]
cloud_filter = wind_filter.loc[wind_filter["Cloudiness"] == 0]
ideal_df = cloud_filter.dropna()

ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [14]:
hotel_df = ideal_df
hotel_df['City'] = hotel_df['City'].str.capitalize() 

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel to {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No Hotel Found... Skipping.")
        
    print("------------")

Closest hotel to Ponta do sol is Hotel Escarpas do Lago.
------------
Closest hotel to Cidreira is Hotel Castelo.
------------
Closest hotel to Morondava is La Case Bambou.
------------
Closest hotel to Vaini is Dandeli Lake County.
------------
Closest hotel to Nador is Hôtel La Giralda.
------------
Closest hotel to Mogok is MOGOK HILL HOTEL.
------------
Closest hotel to Birjand is تاکسی سرویس آرامش.
------------


In [15]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,Ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,88,Cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,Morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,139,Vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,176,Nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
297,297,Mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
444,444,Birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [21]:
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

humid.add_layer(markers)
humid.add_layer(hotel_layer)
humid

Figure(layout=FigureLayout(height='420px'))